In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from aquabyte.visualize import Visualizer
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
import json, os
import cv2
from PIL import Image


In [ ]:
def plot_weight_distribution(weight, mask):
    plt.figure(figsize=(10, 5))
    plt.hist(weight[mask], bins=20)
    plt.grid()
    plt.show()
    
def generate_prediction_stats(weight, mask):
    print('Mean weight estimate: {}'.format(np.mean(weight[mask])))
    print('Median weight estimate: {}'.format(np.median(weight[mask])))
    

In [ ]:
f = '/root/data/alok/biomass_estimation/results/model_lateral_only/results_557ec1732d8bc8bc66951d2ea4e69b935d69b111_model_lateral_only_research-exp-id-02-vikingfjord-20190702-20190708.h5'
df = pd.read_hdf(f, 'table')
df['depth'] = df.world_keypoints.apply(lambda x: x['EYE'][1] if x else None) 
df['difference'] = df.estimated_biomass_g - df.estimated_biomass_g
df1 = df.copy()



In [ ]:
qa_mask = df.is_qa == True
hard_outlier_mask = (df.estimated_biomass_g > 0) & (df.estimated_biomass_g < 20000)
weights = df.estimated_biomass_g.values
plot_weight_distribution(weights, hard_outlier_mask)
generate_prediction_stats(weights, hard_outlier_mask)

In [ ]:
qa_mask = df.is_qa == True
hard_outlier_mask = (df.estimated_biomass_g > 0) & (df.estimated_biomass_g < 20000)
weights = df.estimated_biomass_g.values
plot_weight_distribution(weights, hard_outlier_mask & qa_mask)
generate_prediction_stats(weights, hard_outlier_mask & qa_mask)

In [ ]:
qa_mask = df.is_qa == True
depth_mask = df.depth < 0.75
hard_outlier_mask = (df.estimated_biomass_g > 0) & (df.estimated_biomass_g < 20000)
weights = df.estimated_biomass_g.values
plot_weight_distribution(weights, hard_outlier_mask & depth_mask)
generate_prediction_stats(weights, hard_outlier_mask & depth_mask)

In [ ]:
f = '/root/data/alok/biomass_estimation/results/model_3800_roughly_recovered/results_75ad5ee4527ba2f056de2b8550b9896d372f1805_model_3800_roughly_recovered_research-exp-id-03-vikingfjord-20190709-20190710.h5'
df = pd.read_hdf(f, 'table')
df['depth'] = df.world_keypoints.apply(lambda x: x['EYE'][1] if x else None) 
df['difference'] = df.estimated_biomass_g - df.estimated_biomass_g
df2 = df.copy()

In [ ]:
qa_mask = df.is_qa == True
hard_outlier_mask = (df.estimated_biomass_g > 0) & (df.estimated_biomass_g < 20000)
weights = df.estimated_biomass_g.values
plot_weight_distribution(weights, hard_outlier_mask)
generate_prediction_stats(weights, hard_outlier_mask)

In [ ]:
qa_mask = df.is_qa == True
hard_outlier_mask = (df.estimated_biomass_g > 0) & (df.estimated_biomass_g < 20000)
weights = df.estimated_biomass_g.values
plot_weight_distribution(weights, hard_outlier_mask & qa_mask)
generate_prediction_stats(weights, hard_outlier_mask & qa_mask)

In [ ]:
qa_mask = df.is_qa == True
depth_mask = df.depth < 0.75
hard_outlier_mask = (df.estimated_biomass_g > 0) & (df.estimated_biomass_g < 20000)
weights = df.estimated_biomass_g.values
plot_weight_distribution(weights, hard_outlier_mask & depth_mask)
generate_prediction_stats(weights, hard_outlier_mask & depth_mask)

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')
sql_credentials = json.load(open(os.environ["PROD_SQL_CREDENTIALS"]))
rds_access_utils = RDSAccessUtils(sql_credentials)


In [ ]:
v = Visualizer(rds_access_utils=rds_access_utils, s3_access_utils=s3_access_utils)

In [ ]:
%matplotlib notebook
v.display_3d_keypoints()

In [ ]:
df1['difference'] = df1.estimated_biomass_g - df2.estimated_biomass_g

In [ ]:
%matplotlib inline
mask = df1.difference.abs() < 10000
plt.scatter(df1[mask].depth, df1[mask].difference)
plt.show()

In [ ]:
(df1[mask & (df1.depth < 0.9)].difference / df1[mask & (df1.depth < 0.9)].estimated_biomass_g).mean()

In [ ]:
def enhance(image, clip_limit=5):
    # convert image to LAB color model
    image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # split the image into L, A, and B channels
    l_channel, a_channel, b_channel = cv2.split(image_lab)

    # apply CLAHE to lightness channel
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(8, 8))
    cl = clahe.apply(l_channel)

    # merge the CLAHE enhanced L channel with the original A and B channel
    merged_channels = cv2.merge((cl, a_channel, b_channel))

    # convert image from LAB color model back to RGB color model
    final_image = cv2.cvtColor(merged_channels, cv2.COLOR_LAB2BGR)
    return final_image 

In [ ]:
idx = 60
left_image_url = df1.left_image_url.iloc[idx]
right_image_url = df1.right_image_url.iloc[idx]
left_image_f, _, _ = s3_access_utils.download_from_url(left_image_url)
right_image_f, _, _ = s3_access_utils.download_from_url(right_image_url)
imageL = cv2.imread(left_image_f)
imageR = cv2.imread(right_image_f)

In [ ]:
Image.fromarray(enhance(imageL))

In [ ]:
Image.fromarray(enhance(imageR))

In [ ]:
int(df1.id.iloc[idx])

In [ ]:
%matplotlib inline
v.load_data(int(df1.id.iloc[idx]))
v.display_crops(overlay_keypoints=True, show_labels=False)

In [ ]:
%matplotlib notebook
v.display_3d_keypoints()

In [ ]:
imgL = enhance(imageL)
imgR = enhance(imageR)
good, matchesMask, H = find_matches_and_homography(imgL, imgR)
adjusted_keypoints = adjust_keypoints(keypoints, H)



In [ ]:
df1.difference.iloc[idx]

In [ ]:
import pandas as pd